## html을 통해 한자와 음, 뜻, 자원오행, 음령오행을 추출하고 
## pandas를 이용해 csv 파일을 정리

In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm

In [8]:
# pandas DataFrame
df = pd.DataFrame(columns = [
    'Hanja',
    'mean',
    'sound',
    'meanFiveElements',
    'soundFiveElements'
])

# 한자 추출 후 df에 저장
def hanja_extract(url):
    global df
    response = requests.get(url)
    html_content = response.content
    
    # BeautifulSoup으로 HTML 파싱
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # 특정 문자열이 포함된 텍스트 검색
    results = soup.find_all('form', {'name': "read"})
    
    # 검색된 텍스트 출력
    for result in results:
        result_arr = result.text

    row = pd.DataFrame({'Hanja': [result_arr[1]],
                        'mean': [result_arr[result_arr.find("net")+10:result_arr.find("원획")].split("  ")[0][:-2] + ", " + result_arr[result_arr.find("net")+10:result_arr.find("원획")].split("  ")[1]],
                        'sound': [result_arr[result_arr.find("net")+10:result_arr.find("원획")].split("  ")[0][-1]],
                        'meanFiveElements': [result_arr[result_arr.find("자원오행 :")+6:result_arr.find("음령오행 :")].strip()],
                        'soundFiveElements': [result_arr[result_arr.find("음령오행 :")+6:result_arr.find("상생자음 :")].strip()]})
    
    df = pd.concat([df, row], ignore_index=True)

# 웹페이지 로드
cnt = 0
with open("../hanja_details_urls", 'r') as f:
    for url in tqdm(f):
        hanja_extract(url)
        cnt += 1
        if cnt % 1000 == 0:
            print(cnt)

# csv 파일 생성
df.to_csv('../naming_hanja.csv', index=False)

201it [00:29,  7.24it/s]

200


400it [00:59,  7.41it/s]

400


601it [01:32,  7.15it/s]

...

7801it [20:16,  6.18it/s]

7800


8001it [20:49,  7.34it/s]

8000


8143it [21:14,  6.39it/s]


In [1]:
df = pd.read_csv('../naming_hanja.csv')
print(len(df))

8143


In [3]:
# 좋지 않은 뜻 분류
bad_hanja = df[df['mean'].str.contains('좋지않은')]
print(len(bad_hanja))
bad_hanja.to_csv('../bad_naming_hanja.csv', index=False)

# 좋은 뜻 분류
good_hanja = df[~df['mean'].str.contains('좋지않은')]
print(len(good_hanja))
good_hanja.to_csv('../good_naming_hanja.csv', index=False)

2135
6008


In [ ]:
# 좋은 뜻 한자 filter(불용문자, 성(姓)씨 사용 제거, 한자 첫번째 뜻 사용)
df = pd.read_csv('../good_naming_hanja.csv')

df = df[df['mean'].str.contains('불용문자') == False]
df['soundFiveElements'] = df['soundFiveElements'].astype(str).apply(lambda x: x.replace('성(姓)씨 사용', ''))
df['mean'] = df['mean'].astype(str).apply(lambda x: x.split(',')[0].strip().split(' ')[0])

df.to_csv('../good_naming_hanja_filtered.csv', index=False)